In [1]:
!pip install pandas xlrd ipdb

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
import os
import pandas as pd
import ipdb
from datetime import datetime, time

In [3]:
%ls data

'weekly-2020-02-10 0925.xlsx'


In [4]:
libs = sorted(pd.read_csv('libraries.csv').full_name.tolist())

In [9]:
dn = 'data'
fn = dn + '/' + 'weekly-2020-02-10 0925.xlsx'
xl = pd.ExcelFile(fn)
sheets = xl.sheet_names
res=[]
for sheet in sheets:
    df = xl.parse(sheet)
    date = datetime.strptime(df.columns[0], '%d %B %Y')
    df = (
        df.loc[df['Counter Name'].isin(libs)]
        .rename(columns={df.columns[0]: 'dt'})
        .fillna('x')        
    )
    times=df.loc[df.dt!='x']
    df.dt = df.dt.replace(to_replace='x', method='ffill')
    dt = list(map(lambda x: datetime.combine(date, time(int(x[:2]), 0)), df.dt.tolist()))
    df.dt = dt
    res.append(df)                             

df = pd.concat(res)

,dt,Counter Name,People In,People Out
0,2020-02-03 00:00:00,Chipping Barnet,0,0
6,2020-02-03 00:00:00,Colindale,0,0
9,2020-02-03 00:00:00,East Finchley,0,0
12,2020-02-03 00:00:00,Edgware,0,0
15,2020-02-03 00:00:00,Golders Green,0,0
...,...,...,...,...
561,2020-02-09 23:00:00,East Finchley,0,0
564,2020-02-09 23:00:00,Edgware,0,0
567,2020-02-09 23:00:00,Golders Green,0,0
570,2020-02-09 23:00:00,North Finchley,0,0


In [12]:
df.index

Int64Index([  0,   6,   9,  12,  15,  18,  21,  24,  30,  33,
            ...
            543, 546, 549, 552, 558, 561, 564, 567, 570, 573],
           dtype='int64', length=1176)